In [1]:
import pandas as pd
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import sample

In [2]:
nodes = pd.read_pickle("../../data/mock/mock_nodes.pkl")
channels = pd.read_pickle("../../data/mock/mock_channels.pkl")

In [3]:
nodes.to_csv("../../data/mock/mock_nodes.csv")
channels.to_csv("../../data/mock/mock_channels.csv")

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan


In [6]:
#nodes.reset_index(inplace=True)
#G = nx.DiGraph()
#
## Add nodes to the graph
#for _, node in nodes.iterrows():
#    G.add_node(node['pub_key'], alias=node['alias'])
#
## Add edges to the graph
#for _, edge in channels.iterrows():
#    G.add_edge(edge['node1_pub'], edge['node2_pub'], capacity=edge['capacity'], base_fee=edge['base_fee'], #perc_fee=edge['rate_fee'])
#
## Draw the graph
#pos = nx.spring_layout(G)  # positions for all nodes
#nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_color='black', #font_weight='bold')
#
## Add edge labels
#edge_labels = {(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#
#plt.title('Network Graph')
#plt.show()
#nodes.set_index("pub_key", inplace=True)

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:

def create_demand(pd_object: pd.DataFrame) -> pd.DataFrame:
    """
    This function assigns the role of sender and receiver to
    two random nodes in the network
    :param pd_object: nodes dataframe
    :return: nodes dataset with demand column
    """
    #random.seed(874631)
    counterparties = sample(pd_object.index.to_list(), 2)
    sender = counterparties[0]
    receiver = counterparties[1]
    # Amounts in millisat (aka 10'000'000 is 10'000 sats)
    amount = 1000 #random.randint(a=10000000, b=30000000)

    print(
        f"Transaction of {amount} sats from {pd_object[pd_object.index == sender]['alias'].item()} to {pd_object[pd_object.index == receiver]['alias'].item()}.")

    pd_object["demand"] = 0
    pd_object.loc[pd_object.index == sender, "demand"] = -amount
    pd_object.loc[pd_object.index == receiver, "demand"] = amount

    return pd_object


nodes = create_demand(nodes)



Transaction of 1000 sats from ZMMPQABQVV.com to DVONBCPQBA.com.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [12]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * (1 if channels.loc[(a,b), "base_fee"]<1 else model.x[(a, b)])

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

In [13]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    return sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels']) - sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels']) == nodes.loc[n, "demand"]

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)        

In [14]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())



Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpyt03cg6y.pyomo.lp -stat=1 -solve -solu /tmp/tmpyt03cg6y.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 98 (-690) rows, 687 (-692) columns and 1349 (-696) elements
Statistics for presolved model
Original problem has 689 integers (689 of which binary)
Presolved problem has 1 integers (1 of which binary)
==== 75 zero objective 16 different
==== absolute objective values 16 different
==== for integers 0 zero objective 1 different
1 variables have objective of 1
==== for integers absolute objective values 1 different
1 variables have objective of 1
===== end objective counts


Problem has 98 rows, 687 columns (612 with objective) and 1349 elements
There are 26 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 686 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of typ

In [15]:
ris = []
for i,v in model.a.extract_values().items():
    if v != 0:
        ris.append((i, v))

print(ris)


[(('DVONBCPQBA', 'LCUESNVMDH'), 500.0), (('HLVAPEDHNI', 'INBILMNRAZ'), 500.0), (('LCUESNVMDH', 'PCVFZFFFPN'), 500.0), (('MRUEZMUPCN', 'DVONBCPQBA'), 500.0), (('PCPFRTUCCV', 'MRUEZMUPCN'), 500.0), (('PCVFZFFFPN', 'PCPFRTUCCV'), 500.0), (('TDGQNDRRDO', 'TZBIIOFEHU'), 500.0), (('TDGQNDRRDO', 'ZZVDLVHZVS'), 500.0), (('TZBIIOFEHU', 'HLVAPEDHNI'), 500.0), (('TZBIIOFEHU', 'ZMMPQABQVV'), 1000.0), (('ZZVDLVHZVS', 'INBILMNRAZ'), 500.0)]


In [16]:
from decimal import Decimal
DF = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF.loc[c, "amount"] = Decimal(value)
    DF.loc[c, "source"] = index[0]
    DF.loc[c, "destination"] = index[1]
    c += 1

In [17]:
DF[DF["amount"]>0]

,amount,source,destination
73,500,DVONBCPQBA,LCUESNVMDH
185,500,HLVAPEDHNI,INBILMNRAZ
246,500,LCUESNVMDH,PCVFZFFFPN
267,500,MRUEZMUPCN,DVONBCPQBA
336,500,PCPFRTUCCV,MRUEZMUPCN
349,500,PCVFZFFFPN,PCPFRTUCCV
495,500,TDGQNDRRDO,TZBIIOFEHU
500,500,TDGQNDRRDO,ZZVDLVHZVS
525,500,TZBIIOFEHU,HLVAPEDHNI
529,1000,TZBIIOFEHU,ZMMPQABQVV


In [18]:
nodes.reset_index(inplace=True)
DF_nodes = nodes[nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist()) | nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist())]
nodes.set_index("pub_key", inplace=True)

DF_nodes

,pub_key,alias,addresses,outgoing_channels,incoming_channels,demand
6,PCPFRTUCCV,PCPFRTUCCV.com,1,"[312287859253314262, 821340761473787334, 44023...","[INV312287859253314262, INV821340761473787334,...",0
7,TZBIIOFEHU,TZBIIOFEHU.com,1,"[644952002657532143, 226105421905442682, 79118...","[INV644952002657532143, INV226105421905442682,...",0
9,PCVFZFFFPN,PCVFZFFFPN.com,1,"[622127303179346374, 603656459058207834, 20015...","[INV622127303179346374, INV603656459058207834,...",0
12,ZZVDLVHZVS,ZZVDLVHZVS.com,1,"[776711399907502220, 075145049836483800, 57138...","[INV776711399907502220, INV075145049836483800,...",0
17,DVONBCPQBA,DVONBCPQBA.com,1,"[973105797850423708, 827551619361151868, 44849...","[INV973105797850423708, INV827551619361151868,...",1000
21,HLVAPEDHNI,HLVAPEDHNI.com,1,"[355720426933786839, 960871026467193173, 94158...","[INV355720426933786839, INV960871026467193173,...",0
29,MRUEZMUPCN,MRUEZMUPCN.com,1,"[580046434895452062, 976865912217095860, 03859...","[INV580046434895452062, INV976865912217095860,...",0
51,LCUESNVMDH,LCUESNVMDH.com,1,"[553242481043470896, 519786247212586006, 16730...","[INV553242481043470896, INV519786247212586006,...",0
69,TDGQNDRRDO,TDGQNDRRDO.com,1,"[495357589030588409, 518665765708883840, 22610...","[INV495357589030588409, INV518665765708883840,...",0
